## Visualisation of differences between estimated GHG emissions of hydroelectric reservoirs according to emission factors of Soued et al. and via explicit calculation with G-res

#### Author: Tomasz Janus
#### Date: 06/11/24

In [ ]:
# Import required libraries
from typing import List, Set, Any, Optional
import pathlib
import os
import numpy as np
import pandas as pd
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties
from pypalettes import load_cmap
# Set the required constants and paths
font_path = pathlib.Path('custom_fonts/Urbanist-main/fonts/ttf/Urbanist-Thin.ttf')
bold_font_path = pathlib.Path('custom_fonts/Urbanist-main/fonts/ttf/Urbanist-Medium.ttf')
from matplotlib.colors import Normalize

In [ ]:
fpath = os.path.join(matplotlib.get_data_path(), font_path.absolute())
fpath_bold = os.path.join(matplotlib.get_data_path(), bold_font_path.absolute())
prop = FontProperties(fname=fpath)
prop_bold = FontProperties(fname=fpath_bold)
plot_fonts = False
prop_legend = FontProperties(fname=fpath, size=13)
prop_title = FontProperties(fname=fpath_bold, size= 16)

In [ ]:
def plot_installed_fonts() -> HTML:
    def make_html(fontname):
        return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)
    code = "\n".join([make_html(font) for font in sorted(set([f.name for f in font_manager.fontManager.ttflist]))])
    return HTML("<div style='column-count: 2;'>{}</div>".format(code))
if plot_fonts:
    installed_plots = plot_installed_fonts()

In [ ]:
dam_data_filename = pathlib.Path("outputs/moo/all_hp.csv")
dam_data: pd.DataFrame = (
    pd.read_csv(dam_data_filename).
    drop(columns=['Unnamed: 0']).
    assign(tot_em = lambda x: x['tot_em'] / 1_000).
    assign(tot_em_soued = lambda x: x['tot_em_soued'] / 1_000)
)
dam_data['em_diff'] = dam_data.tot_em_soued - dam_data.tot_em
# Restrict our analysis only to storage dams, i.e. exclude RoR for which emissions are equal in
# Soued's and ReEmission's estimates.
dam_data_sto = dam_data.query("hp_type_reem == 'sto'") 	

In [ ]:
# Check data classification - hp_type_reem != sto should coincide with tot_em == tot_em_soued
ror_units = dam_data.query("hp_type_reem != 'sto'") 
assert (any(ror_units.tot_em - ror_units.tot_em_soued) is False)

In [ ]:
font=font_path
bold_font = bold_font_path
def make_stem_plot(df: pd.DataFrame) -> None:
    cmap = load_cmap('Coconut', cmap_type='continuous', reverse=True)
    fig, ax = plt.subplots(figsize=(10,15), dpi=300)
    df_sorted = df.sort_values(by=['tot_em'], ascending=True)
    names = df_sorted['name']
    em_diff = df_sorted['em_diff']
    hp_factor = 0.7
    hp = df_sorted['HP_mean'] * hp_factor
    tot_em = df_sorted['tot_em']
    min_col = tot_em.min()
    max_col = tot_em.max()
    norm = Normalize(vmin=min_col, vmax=max_col)
    scatter = ax.scatter(
        marker='o', y=names, x = em_diff, c = tot_em, cmap=cmap, 
        edgecolor='k', s = hp, linewidth=0.4, alpha=0.8)
    cbar = plt.colorbar(
        scatter, shrink=0.7, label="GHG emissions, ktCO$_{2e}$/year", 
        pad=0.03, aspect=30)
    cbar.outline.set_linewidth(0.5)
    cbar.set_ticklabels(cbar.ax.get_yticklabels(), font=font)
    cbar.set_label("GHG emissions, ktCO$_{2e}$/year", fontsize=14, font=font)
    # Add stems
    for _, row in df_sorted.iterrows():
        name = row['name']
        em_diff = row['em_diff']
        color = cmap(row['tot_em'])
        normalized_col = norm(row['tot_em'])
        stems = ax.plot(
            [0, em_diff], [name, name], 
            c=cmap(normalized_col), alpha=0.8, linewidth=0.8)
    
    v_lines = [-200, 0, 200, 400, 600]
    ax.vlines(
        x=v_lines, ymin=0, ymax=len(df_sorted)-1, colors=[cmap(val) for val in v_lines], 
        linewidth=1.2, zorder=-1, alpha=0.5)
    ax.spines[['left', 'right', 'top', 'bottom']].set_visible(False)
    ax.tick_params(axis=u'both', which=u'both',length=0)
    plt.setp(ax.get_xticklabels(), visible=False)
    ax.yaxis.set_tick_params(labelsize=4)
    for value in v_lines:
       ax.text(
           y=-1, x=value, s=f'{value}', color=cmap(value), 
           size=14, va='top', ha='center', font=font)

    for hp_value in [1, 100, 500, 2000, 4000]:  # Adjust these values as necessary for your range
        plt.scatter(
            [], [], marker = "o", s=hp_factor * hp_value, color='grey', 
            alpha=0.45, label=f'{hp_value} MW')

    leg = plt.legend(
        scatterpoints=1, frameon=False, 
        labelspacing=1.5, title="Mean Power Output", 
        fontsize=15, prop=prop_legend, loc="right")
    leg.set_title("Mean Power Output", prop={'size': 14})
    title = plt.title(
        'Difference in Net Anthropogenic Emissions: Emission Factors vs. Explicit Calculation', 
        fontsize=15, 
        fontproperties=prop_title)
    plt.xlabel(
        'Calculated Emission Difference (ktCO$_{2e}$/year): Emission Factors - Explicit Calculation', 
        fontsize=15, font=font)

    for tick in ax.get_xticklabels():
        tick.set_fontproperties(prop)
        
    for tick in ax.get_yticklabels():
        tick_label = tick.get_text()
        status = df[df['name'] == tick_label]['status'].iloc[0]
        tick.set_fontproperties(prop)
        tick.set_visible(True)
        if status == "Future":
            tick.set_color('black')  # Set font weight to bold
        else:
            tick.set_color('tab:blue')  # Set font weight to bold
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    plt.savefig(
        "figures/emission_comparison/hp_total_emission_comparison.pdf", 
        format="pdf", bbox_inches="tight")
    plt.savefig(
        "figures/emission_comparison/hp_total_emission_comparison.svg", 
        format="svg", bbox_inches="tight")
    plt.savefig(
        "figures/emission_comparison/hp_total_emission_comparison.png", 
        format="png", bbox_inches="tight")
    plt.show()

make_stem_plot(dam_data_sto)